In [1]:


# This is lzl's snippet for diploma project
import math
import os,shutil
import random
import warnings
import gc

import geopandas as gpd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shapely
import seaborn as sns
from shapely.geometry import LineString, Point, Polygon
from shapely.wkt import dumps, loads
from tqdm import tqdm

import traProject as tp
from traProject.analysisFuns import statisticFuns as sfuns
from traProject.basicFuns import CalFuns as cfun,VisFuns as vfun,DataProcessingFuns as dfun
from traProject.classificationFuns import models
import traProject.utils as tu

random.seed(1229)

plt.style.use('seaborn')
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

warnings.filterwarnings("ignore")



In [7]:
path = '../../data/wuhan/WH_GPS/WH-201908'
path_read = []
def check_if_dir(file_path):
    temp_list = os.listdir(file_path)
    if temp_list and os.path.isfile(file_path + '/' + temp_list[0]): 
        for temp_list_each in temp_list:
            temp_path = file_path + '/' + temp_list_each
            if os.path.splitext(temp_path)[-1] == '.csv':
                path_read.append(temp_path)
            else:
                continue
    else:
        for temp_list_each in temp_list:
            check_if_dir(file_path + '/' + temp_list_each)

In [9]:
if path_read:
    path_read.clear()
check_if_dir(path)
print('num of csv data is: {}'.format(len(path_read)))
path_read
x=pd.DataFrame(path_read)
x.to_csv(outpath+'/path.csv')

num of csv data is: 330624


In [11]:
for idx,filepath in enumerate(path_read):
    tmp=pd.read_csv(filepath)
    if idx==0:
        tmp.to_csv(outpath+'/merged.csv',index=0)
    else:
        tmp.to_csv(outpath+'/merged.csv',index=0,mode='a',header=None)

In [3]:
test=pd.read_csv(outpath+'/merged.csv',usecols=['IMEIorMEID','FIX_TIME'])
test

,IMEIorMEID,FIX_TIME
0,159,1.564154e+09
1,159,1.564179e+09
2,159,1.564192e+09
3,159,1.564198e+09
4,159,1.564198e+09
...,...,...
45471726,6172,1.566944e+12
45471727,6173,1.567172e+12
45471728,6174,1.567141e+12
45471729,6174,1.567207e+12


In [4]:
test=tu.calDate(test,timeindex='FIX_TIME',unit='ms',calList=['year','month','day'])

In [5]:
test.groupby(['year','month']).count()

IMEIorMEID  FIX_TIME       day
year   month                                
1970.0 1.0        130873    130873    130873
1999.0 12.0        24144     24144     24144
2000.0 1.0         21335     21335     21335
       12.0           89        89        89
2001.0 1.0             3         3         3
2018.0 8.0            11        11        11
       11.0          441       441       441
       12.0         1015      1015      1015
2019.0 1.0          3642      3642      3642
       2.0          7744      7744      7744
       3.0         25492     25492     25492
       4.0         43849     43849     43849
       5.0        133410    133410    133410
       6.0        518410    518410    518410
       7.0      12384161  12384161  12384161
       8.0      32091836  32091836  32091836
       9.0         82937     82937     82937
       11.0            8         8         8
       12.0            2         2         2
2020.0 3.0             7         7         7
       5.0             2         2         2
       7.0             2         2         2
       12.0            1         1         1
2021.0 12.0            2         2         2
2022.0 3.0             1         1         1
2033.0 9.0             1         1         1
2037.0 8.0             1         1         1

# 时间戳正确性

In [19]:
def check_if_dir(file_path):
    temp_list = os.listdir(file_path)
    if temp_list and os.path.isfile(file_path + '/' + temp_list[0]): 
        for temp_list_each in temp_list:
            temp_path = file_path + '/' + temp_list_each
            if os.path.splitext(temp_path)[-1] == '.csv':
                path_read.append(temp_path)
            else:
                continue
    else:
        for temp_list_each in temp_list:
            check_if_dir(file_path + '/' + temp_list_each)

In [24]:
def myMerge(path,outpath,fname):
    if path_read:
        path_read.clear()
    check_if_dir(path)
    for idx,filepath in enumerate(path_read):
        tmp=pd.read_csv(filepath)
        if idx==0:
            tmp.to_csv(outpath+'/%s_merged.csv'%fname,index=0)
        else:
            tmp.to_csv(outpath+'/%s_merged.csv'%fname,index=0,mode='a',header=None)

In [21]:
flist=tu.getFileNames('../../data/wuhan/WH_GPS/WH-201908/')

In [28]:
for f in flist:
    myMerge(f,tu.pathCheck('../../output/tmp/时间戳问题排查/step1/'),f.split('/')[-1])

In [40]:
def myMerge1(path,outpath):
    path_read=tu.getFileNames(path)
    for f1 in path_read:
        tmp=pd.read_csv(f1)
        fname=f1.split('/')[-1].split('_')[0]+'.csv'
        opath=outpath+fname
        if not os.path.exists(opath):
            tmp.to_csv(opath,index=0)
        else:
            tmp.to_csv(opath,index=0,mode='a',header=None)

In [41]:
myMerge1('../../output/tmp/时间戳问题排查/step1/','../../output/tmp/时间戳问题排查/step2/')

D:\progaramData\anaconda\envs\tra\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [57]:
flist2=tu.getFileNames('../../output/tmp/时间戳问题排查/step2/')
ANS=[]
for f2 in tqdm(flist2):
    fdate=f2.split('/')[-1].split('.')[0]
    fmonth=int(fdate.split('-')[1])
    fday=int(fdate.split('-')[2])
    data=pd.read_csv(f2)
    data=tu.calDate(data,timeindex='FIX_TIME',unit='ms',calList=['month','day'])
    sum1=len(data)
    sum2=len(data[(data.month==fmonth)&(data.day==fday)])
    tmp=[fdate,sum1,sum2]
    ANS.append(tmp)

  0%|                                                 | 0/38 [00:00<?, ?it/s]D:\progaramData\anaconda\envs\tra\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
  3%|█                                        | 1/38 [00:00<00:23,  1.55it/s]D:\progaramData\anaconda\envs\tra\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
  5%|██▏                                      | 2/38 [00:02<00:53,  1.49s/it]D:\progaramData\anaconda\envs\tra\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
  8%

 66%|██████████████████████████▎             | 25/38 [00:44<00:22,  1.75s/it]D:\progaramData\anaconda\envs\tra\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
 68%|███████████████████████████▎            | 26/38 [00:46<00:20,  1.74s/it]D:\progaramData\anaconda\envs\tra\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
 71%|████████████████████████████▍           | 27/38 [00:48<00:19,  1.74s/it]D:\progaramData\anaconda\envs\tra\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
 74%

In [59]:
ANS

[['2019-07-30', 426702, 106],
 ['2019-07-31', 1331876, 2096],
 ['2019-08-01', 1349045, 2211],
 ['2019-08-02', 1206775, 2318],
 ['2019-08-03', 1355242, 2352],
 ['2019-08-04', 1361739, 1276],
 ['2019-08-05', 1255911, 1525],
 ['2019-08-06', 1335105, 2114],
 ['2019-08-07', 1520234, 1879],
 ['2019-08-08', 1198543, 885],
 ['2019-08-09', 1229471, 628],
 ['2019-08-10', 1208576, 572],
 ['2019-08-11', 1064962, 589],
 ['2019-08-12', 922962, 268],
 ['2019-08-13', 1289667, 1674],
 ['2019-08-14', 1315902, 1643],
 ['2019-08-15', 1184830, 1443],
 ['2019-08-16', 963287, 8215],
 ['2019-08-17', 1233277, 758],
 ['2019-08-18', 1284475, 1470],
 ['2019-08-19', 1206429, 10177],
 ['2019-08-20', 1182755, 1315],
 ['2019-08-21', 1266890, 1102],
 ['2019-08-22', 1221199, 670],
 ['2019-08-23', 1150888, 1599],
 ['2019-08-24', 1192853, 1058],
 ['2019-08-25', 1205961, 1300],
 ['2019-08-26', 497655, 186],
 ['2019-08-27', 1102688, 1001],
 ['2019-08-28', 1096238, 41398],
 ['2019-08-29', 1194772, 1363],
 ['2019-08-30', 128

In [66]:
pd.DataFrame(ANS,columns=['文件日期','数据量','匹配数据量']).to_csv('../../output/tmp/时间戳问题排查/分析结果1.csv')

In [76]:
flist2=tu.getFileNames('../../output/tmp/时间戳问题排查/step2/')
for f2 in tqdm(flist2):
    fdate=f2.split('/')[-1].split('.')[0]
    fmonth=int(fdate.split('-')[1])
    fday=int(fdate.split('-')[2])
    data=pd.read_csv(f2)
    data=tu.calDate(data,timeindex='FIX_TIME',unit='ms',calList=['month','day'])
    tmp=data.groupby(['month','day']).IMEIorMEID.count()
    tmp.sort_values(ascending=False).to_csv(tu.pathCheck('../../output/tmp/时间戳问题排查/分析结果2/')+'%s.csv'%fdate)

  0%|                                                 | 0/38 [00:00<?, ?it/s]D:\progaramData\anaconda\envs\tra\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
100%|████████████████████████████████████████| 38/38 [01:15<00:00,  1.99s/it]
